In [1]:
import pandas as pd
import numpy as np

In [3]:
import pyportopt as opt

In [4]:
from pandas_datareader import data

In [8]:
"""Define parameters for data request."""

tickers = ['AGG', 'EBND', 'URTH', 'EEM']
data_source = 'yahoo'
start_date = '2014-03-31'
end_date = '2018-03-31'

In [9]:
"""Request data, ensure sorted by ascending date, and normalise."""

panel_data = data.DataReader(tickers, data_source, start_date, end_date)
px_data = panel_data.loc['Adj Close'].dropna().sort_index(ascending=True)
px_data = px_data / px_data.iloc[0]

In [10]:
"""
TEST CASE
Add constraints: Portfolio weights must sum to 100%,
Add boundaries: Long only, no short positions.
"""

cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
]

bnds = [(0, 1) for x in px_data.columns]

In [12]:
"""
Define the relaxation tolerance allowed for init_func.
For example, if prim_func is max_er and init_func is max_dr,
relax_tolderance=0.1 and num_of_steps=10 results in a tolerance of 10% from max_dr optimal in favour of max_er,
performed in 10 steps.
"""

relax_tolerance = 0.1
num_of_steps = 10

In [15]:
"""Run optimisation."""

opt_results = opt.dual_target_optimisation(
    opt.max_er, opt.min_vol, px_data, 
    relax_tol=relax_tolerance, steps=num_of_steps,
    rf=0.0, scaling_fact=252, 
    constraints=cons, bounds=bnds)

Total time: 3.2692368030548096 secs


In [16]:
"""View results."""

opt_results

,max_er,min_vol,AGG,EBND,EEM,URTH
step,,,,,,
0,0.025965,0.028787,0.9170,0.0,0.0,0.0830
1,0.027516,0.029107,0.8874,0.0,0.0,0.1126
2,0.028167,0.029427,0.8749,0.0,0.0,0.1251
3,0.028671,0.029747,0.8653,0.0,0.0,0.1347
4,0.029100,0.030066,0.8572,0.0,0.0,0.1428
5,0.029481,0.030386,0.8499,0.0,0.0,0.1501
6,0.029828,0.030706,0.8433,0.0,0.0,0.1567
7,0.030149,0.031026,0.8372,0.0,0.0,0.1628
8,0.030451,0.031346,0.8314,0.0,0.0,0.1686
